In [1]:
%run nb_gold_functions

StatementMeta(, 01822373-d4a6-4204-8142-7c065b59e4e6, 8, Finished, Available, Finished)

In [2]:
workspace = 'ETL_Paycom_DEV'
lakehouse = 'lh_paycom_silver'

StatementMeta(, 01822373-d4a6-4204-8142-7c065b59e4e6, 9, Finished, Available, Finished)

In [3]:
startTime = dt.now()
#syncs the sql endpoint so any report refreshes will grab the latest data
sync_result = udf_SyncSqlEndpoint(workspace,lakehouse)


StatementMeta(, 01822373-d4a6-4204-8142-7c065b59e4e6, 10, Finished, Available, Finished)

Table: HCM_Rosters   Last Update: 2025-02-26T17:10:42.2788451Z  Table Sync State: NotRun  SQL Sync State: NotRun   Table Warnings: []
Table: Zipline_User_File_HQ   Last Update: 2025-02-25T21:24:56.4134Z  Table Sync State: Failure  SQL Sync State: Failure   Table Warnings: []
Table: Legion_Daily_Punches   Last Update: 2025-01-30T16:27:52.7700882Z  Table Sync State: Failure  SQL Sync State: Failure   Table Warnings: []
Table: Driving_Status   Last Update: 2025-02-25T21:24:57.1302043Z  Table Sync State: Failure  SQL Sync State: Failure   Table Warnings: []
Table: Zipline_User_File_SFTP   Last Update: 2025-02-25T21:24:56.91145Z  Table Sync State: Failure  SQL Sync State: Failure   Table Warnings: []
Table: Accrual_Report_for_Legion   Last Update: 2025-02-25T21:24:56.0520114Z  Table Sync State: Failure  SQL Sync State: Failure   Table Warnings: []
Table: EE_Code_and_Name   Last Update: 2025-02-26T15:10:12.9696175Z  Table Sync State: Failure  SQL Sync State: Failure   Table Warnings: []
Tabl

In [4]:
# Extract function
def parse_log_entry(entry):
    return {
        "tableName": re.search(r"Table: (.*?)\s{2}", entry).group(1),
        "lastSuccessfulUpdate": re.search(r"Last Update: (.*?)\s{2}", entry).group(1),
        "tableSyncState": re.search(r"Table Sync State: (.*?)\s{2}", entry).group(1),
        "sqlSyncState": re.search(r"SQL Sync State: (.*?)\s{2}", entry).group(1),
        "warningMessages": re.search(r"Table Warnings: (.*)", entry).group(1)
    }

# Convert list of log entries to structured data
parsed_data = [parse_log_entry(entry) for entry in sync_result]

# Define schema
schema = StructType([
    StructField("tableName", StringType(), True),
    StructField("lastSuccessfulUpdate", StringType(), True),
    StructField("tableSyncState", StringType(), True),
    StructField("sqlSyncState", StringType(), True),
    StructField("warningMessages", StringType(), True)
])

# Convert list to DataFrame
df = spark.createDataFrame(parsed_data, schema=schema)

# Show DataFrame
#df.show(truncate=False)

df_filtered = df.where(df["tableSyncState"] != "NotRun")
#df_filtered.show(truncate=False)
# Collect filtered table names and sync states into a list
table_list = df_filtered.select("tableName", "tableSyncState").collect()

# Convert list of Row objects to formatted strings
table_strings = [f"{row['tableName']} (Table Sync State: {row['tableSyncState']})" for row in table_list]

# Join the list into a single string
result_string = ", ".join(table_strings)

# Print final output
print(result_string)


StatementMeta(, 01822373-d4a6-4204-8142-7c065b59e4e6, 11, Finished, Available, Finished)

Zipline_User_File_HQ (Table Sync State: Failure), Legion_Daily_Punches (Table Sync State: Failure), Driving_Status (Table Sync State: Failure), Zipline_User_File_SFTP (Table Sync State: Failure), Accrual_Report_for_Legion (Table Sync State: Failure), EE_Code_and_Name (Table Sync State: Failure), HCM_Rosters_HR_Regionals (Table Sync State: Failure), Retail_Availability_Report_Legion (Table Sync State: Failure), Legion_Garnishment_Report (Table Sync State: Failure)


In [5]:
# Return output to caller for logging
details = f'Synced {result_string} tables in the {lakehouse} lakehouse within {workspace} workspace.'
stopTime = dt.now()
returnVal = {
    'startTime': str( startTime ),
    'stopTime': str( stopTime ),
    'details': str( details )
}
notebookutils.notebook.exit( returnVal )

StatementMeta(, 01822373-d4a6-4204-8142-7c065b59e4e6, 12, Finished, Available, Finished)

ExitValue: {'startTime': '2025-02-26 19:09:30.083561', 'stopTime': '2025-02-26 19:09:39.223927', 'details': 'Synced Zipline_User_File_HQ (Table Sync State: Failure), Legion_Daily_Punches (Table Sync State: Failure), Driving_Status (Table Sync State: Failure), Zipline_User_File_SFTP (Table Sync State: Failure), Accrual_Report_for_Legion (Table Sync State: Failure), EE_Code_and_Name (Table Sync State: Failure), HCM_Rosters_HR_Regionals (Table Sync State: Failure), Retail_Availability_Report_Legion (Table Sync State: Failure), Legion_Garnishment_Report (Table Sync State: Failure) tables in the lh_paycom_silver lakehouse within ETL_Paycom_DEV workspace.'}